In [ ]:
def show_results(outputs):
    for key in ['retrieved', 'output_parameters', 'output_structure', 'band_structure']:
        if key in outputs:
            display(outputs[key])

In [ ]:
import os
import threading
import tempfile
from enum import Enum
from pprint import pprint
from time import sleep
from collections import deque

import ase
import nglview
import ipywidgets as ipw
import traitlets

from aiidalab_widgets_base import CodeDropdown, ProgressBarWidget

from aiida.engine import submit
from aiida.orm import load_node, Dict, Float,Str, StructureData, WorkChainNode
from aiida.plugins import WorkflowFactory


DEFAULT_PARAMETERS = Dict(dict={"SYSTEM": {"ecutwfc": 50., "ecutrho": 200.}})


class StagesApp(ipw.VBox):
    
    _ICON_SEPARATOR = '\u2000'  # en-dash  (placed between title and icon)
    _ICON_SUCCESS = '\u2714'    # ✔
    _ICON_FAIL = '\u2715'       # ✕
            
    class State(Enum):
        "Every stage within the StagedApp must have this traitlet."
        INIT = 0  # implicit default value
        
        CONFIGURED = 1
        READY = 2
        ACTIVE = 3
        SUCCESS = 4
        
        # All error states have negative codes
        FAIL = -1

    def __init__(self, stages, **kwargs):
        # The number of stages must be greater than one
        # for this app's logic to make sense.
        assert len(stages) > 1
        self.stages = stages
        
        # Unzip the stages to titles and widgets.
        titles, widgets = zip(*stages)
        
        # Watch for changes to each stage's state
        for widget in widgets:
            assert widget.has_trait('state') and widget.state == self.State.INIT
            widget.observe(self._update_stage_state, names=['state'])

        # Initialize the accordion with the widgets ...
        self.accordion = ipw.Accordion(children=widgets)
        # ... and set the enumerated titles:
        for i, title in enumerate(titles):
            self.accordion.set_title(i, f"{i+1}. {title}{self._ICON_SEPARATOR}")
        # Activate/deactivate the next-button based on which stage is selected.
        self.accordion.observe(self._observe_stage_change, names='selected_index')

        # Create a next-button, to switch to the next stage when appropriate:
        self.next_button = ipw.Button(
            description='Next',
            icon='step-forward',
            layout=ipw.Layout(width='auto', flex='1 1 auto'),
            disabled=True)
        self.next_button.on_click(self._go_to_next_stage)
    
        super().__init__(children=[self.accordion, self.next_button], **kwargs)
            
    def _set_title_icon(self, index, icon):
        "Set the title icon of the stage with index."
        tmp = self.accordion.get_title(index).rpartition(self._ICON_SEPARATOR)
        self.accordion.set_title(index, tmp[0] + tmp[1] + icon)        
        
    def _update_stage_state(self, change):
        widget = change['owner']
        widget_index = self.accordion.children.index(widget)
        
        if change['new'] == self.State.SUCCESS:
            # The stage was successful, set icon and ...
            self._set_title_icon(widget_index, self._ICON_SUCCESS)
            
            # ... request_ready_state next stage to enter ready state.
            if self.accordion.selected_index < len(self.accordion.children) -1:
                self.accordion.children[self.accordion.selected_index+1].request_ready_state()
                       
#             # Directly open the next stage (deactivated).
#             self.accordion.selected_index = min(len(self.accordion.children)-1, widget_index+1)
            
            # Activate the next-button:
            self.next_button.disabled = False
            
        elif change['new'] == self.State.FAIL:
            # The stage failed, set icon and do nothing.
            self._set_title_icon(widget_index, self._ICON_FAIL)
            self.next_button.disabled = True
        else:
            self.next_button.disabled = True
            
    def _observe_stage_change(self, change):
        widget = self.accordion.children[self.accordion.selected_index]
        self.next_button.disabled = widget.state != self.State.SUCCESS
        
    def _go_to_next_stage(self, _):
        self.accordion.selected_index = min(
            len(self.accordion.children)-1,
            self.accordion.selected_index+1)


class StagesAppStage(traitlets.HasTraits):
        
    state = traitlets.UseEnum(StagesApp.State)
    
    # The state of the stage can no longer be changed once it is in a sealed state.
    SEALED_STATES = [
        StagesApp.State.SUCCESS,
        StagesApp.State.FAIL,
    ]
    
    @traitlets.default('state')
    def _default_state(self):
        return StagesApp.State.INIT
    
    @traitlets.validate('state')
    def _valid_state_transition(self, proposal):
        with self.hold_trait_notifications():
            old, new = self.state, proposal['value']

            if old != new and old in self.SEALED_STATES:
                raise traitlets.TraitError("Unable to change state when previous state is SUCCESS or FAIL.")

            return new


class StructureFileUploadWidget(ipw.VBox):
    
    file = traitlets.Tuple(traitlets.Unicode(), traitlets.Bytes())

    def __init__(self):
        self.description = ipw.Label("Upload a structure file:")
        self.supported_formats = ipw.HTML(
            """<a href="https://wiki.fysik.dtu.dk/ase/_modules/ase/io/formats.html" target="_blank">
            Supported formats
            </a>""")
        self.file_upload = ipw.FileUpload(
            description="Select file",
            multiple=False,
            )
        self.file_upload.observe(self.on_file_uploaded, names='value')
        
        super().__init__(children=[self.description, self.file_upload, self.supported_formats])
        
    def on_file_uploaded(self, change):
        for fn, item in change['new'].items():
            self.file = (fn, item['content'])
            break
            
    def freeze(self):
        self.file_upload.disabled = True
            

class SelectionStructureUploadWidget(ipw.Dropdown):
    
    file = traitlets.Tuple(traitlets.Unicode(), traitlets.Bytes())
    
    def __init__(self, options=None, hint='Select structure', **kwargs):
        if options is None:
            options = []
        else:
            options.insert(0, (hint, None))
            
        super().__init__(options=options, **kwargs)
        
        self.observe(self._on_select, 'index')
        
    def _on_select(self, change):
        assert change['name']  == 'index'
        fn_selected = self.options[change['new']][1]
        with open(fn_selected, 'rb') as file:
            self.file = fn_selected, file.read()
            
    def reset(self):
        self.file = (None, None)
        self.index = 0
            
    def freeze(self):
        self.disabled = True


class StructureUploadComboWidget(ipw.VBox, StagesAppStage):

    structure = traitlets.Instance(ase.atoms.Atoms, allow_none=True)   
    confirmed_structure = traitlets.Instance(ase.atoms.Atoms, allow_none=True)
    
    def __init__(self, data_importers=None, examples=None, viewer=True, **kwargs):        
        if data_importers is None:
            self.data_importers = [('Upload', StructureFileUploadWidget())]
        else:
            self.data_importers = data_importers
            
        if examples:
            self.example_widget = SelectionStructureUploadWidget(options=examples)
            self.data_importers.append(('Examples', self.example_widget))
        else:
            self.example_widget = None
            
        if len(self.data_importers) > 1:
            self.structure_sources_tab = ipw.Tab(children=[s[1] for s in self.data_importers])
            for i, source in enumerate(self.data_importers):
                self.structure_sources_tab.set_title(i, source[0])
        else:
            self.structure_sources_tab = self.data_importers[0][1]
            
        self.structure_sources_tab.layout = ipw.Layout(
            display='flex',
            flex_flow='column',
        )
            
        for data_importer in self.data_importers:
            data_importer[1].observe(self._on_structure_file_selection, 'file')
            
        if viewer:
            self.viewer = nglview.NGLWidget(width='300px', height='300px')
            self.viewer_box = ipw.Box(children=[self.viewer], layout=ipw.Layout(border='solid 1px'))
        else:
            self.viewer = None
            self.viewer_box = None
            
        self.structure_name_text = ipw.Text(
            placeholder='Structure',
            description='Selected:',
            disabled=True,
        )
        
        self.structure_sources_tab.layout = ipw.Layout(min_width='600px')
        
        grid = ipw.GridspecLayout(1, 3)
        grid[:, :2] = self.structure_sources_tab
        grid[:, 2] = self.viewer_box
        
        self.confirm_button = ipw.Button(
            description='Confirm',
            button_style='success',
            icon='check-circle',
            disabled=True,
            layout=ipw.Layout(width='auto'),
        )
        self.confirm_button.on_click(self.confirm)
       
        super().__init__(
            children=[grid, self.structure_name_text, self.confirm_button], **kwargs)
        
    @traitlets.observe('state')
    def _observe_state(self, change):
        self.confirm_button.disabled = not self.state is StagesApp.State.READY
        
    @traitlets.observe('structure')
    def _observe_structure(self, change):
        structure = change['new']
        if structure is None:
            self.confirm_button.disabled = True
            self.structure_name_text.value = ""
        else:
            self.structure_name_text.value = str(self.structure.get_chemical_formula())
            self.confirm_button.disabled = False
        self.refresh_view()
        
    def _on_structure_file_selection(self, change):
        assert change['name'] == 'file'
        fn, data = change['new']
        basename, ext = os.path.splitext(fn)
        try:
            with tempfile.NamedTemporaryFile(suffix=ext) as file:
                file.write(data)
                file.flush()
                
                traj = ase.io.read(file.name, index=':')
            # TODO: REACT TO len(traj) > 1
        except Exception as error:
            print(error)
        else:
            self.structure = traj[0]
            
    def refresh_view(self):
        # Note: viewer.clear() only removes the 1st component (TODO: FIX UPSTREAM!)
        for comp_id in self.viewer._ngl_component_ids:
            self.viewer.remove_component(comp_id)
        if self.structure is not None:
            self.viewer.add_component(nglview.ASEStructure(self.structure))
            self.viewer.add_unitcell()
                         
    def confirm(self, button):
        for child in self.structure_sources_tab.children:
            child.freeze()
        self.confirm_button.disabled = True
        self.confirm_button.icon = 'check-circle'
        self.confirmed_structure = self.structure
        self.state = StagesApp.State.SUCCESS

    
def get_calc_job_output(process):
    from aiidalab_widgets_base.process import get_running_calcs
    previous_calc_id = None
    num_lines = 0
    
    while not process.is_sealed:
        calc = None
        for calc in get_running_calcs(process):
            if calc.id == previous_calc_id:
                break
        else:
            if calc:
                previous_calc_id = calc.id

        if calc and 'remote_folder' in calc.outputs:
            f_path = os.path.join(calc.outputs.remote_folder.get_remote_path(), calc.attributes['output_filename'])
            if os.path.exists(f_path):
                with open(f_path) as fobj:
                    new_lines = fobj.readlines()[num_lines:]
                    num_lines += len(new_lines)
                    yield from new_lines
                      

class ProgressBarWidget(ipw.VBox):
    """A bar showing the proggress of a process."""

    def __init__(self, **kwargs):
        self._process = None
        self.correspondance = {
            None: (0, 'warning'),
            "created": (0, 'info'),
            "running": (1, 'info'),
            "waiting": (1, 'info'),
            "killed": (2, 'danger'),
            "excepted": (2, 'danger'),
            "finished": (2, 'success'),
        }
        self.bar = ipw.IntProgress(  # pylint: disable=blacklisted-name
            value=0,
            min=0,
            max=2,
            step=1,
#             description='Progress:',
            bar_style='warning',  # 'success', 'info', 'warning', 'danger' or ''
            orientation='horizontal',
            layout=ipw.Layout(width="auto")
        )
        self.state = ipw.HTML(
            description="Calculation state:", value='',
            style={'description_width': '100px'},
        )
        super().__init__(children=[self.state, self.bar], **kwargs)

    def update(self):
        """Update the bar."""
        self.bar.value, self.bar.bar_style = self.correspondance[self.current_state]
        if self.current_state is None:
            self.state.value = 'N/A'
        else:
            self.state.value = self.current_state.capitalize()
        
    @property
    def process(self):
        return self._process
    
    @process.setter
    def process(self, value):
        self._process = value
        self.update()

    @property
    def current_state(self):
        if self.process is not None:
            return self.process.process_state.value


class LogOutputWidget(ipw.VBox):
    
    def __init__(self, title='Output:', num_lines_shown=3, **kwargs):
        self.description = ipw.Label(value=title)
        self.last_lines = ipw.HTML()
        
        self.lines = []
        self.lines_shown = deque([''] * num_lines_shown, maxlen=num_lines_shown)
        
        self.raw_log = ipw.Textarea(
            layout=ipw.Layout(
                width='auto',
                height='auto',
                display='flex',
                flex='1 1 auto',
            ),
            disabled=True)

        self.accordion = ipw.Accordion(children=[self.raw_log])
        self.accordion.set_title(0, 'Raw log')
        self.accordion.selected_index = None

        self.update()        
        super().__init__(children=[self.description, self.last_lines, self.accordion], **kwargs)
        
    def clear(self):
        self.lines = []
        self.lines_shown.extend('' * len(self.lines_shown))
        self.update()
        
    def format_code(self, text):
        return '<pre style="background-color: #1f1f2e; color: white;">{}</pre>'.format(text)
        
    def append_line(self, line):
        self.lines.append(line.strip())
        self.lines_shown.append("{:03d}: {}".format(len(self.lines), line.strip()))
        self.update()
        
    def update(self):
        self.last_lines.value = self.format_code('\n'.join(self.lines_shown))
        self.raw_log.value = '\n'.join(self.lines)
            
            
class ProcessStatusWidget(ipw.VBox):

    def __init__(self, **kwargs):
        self._process = None
        self._process_output = None
        self.progress_bar = ProgressBarWidget()
        self.log_output = LogOutputWidget()

        super().__init__(children=[self.progress_bar, self.log_output], **kwargs)
        
    @property
    def process(self):
        return self._process
    
    @process.setter
    def process(self, value):
        self._process = value
        self._process_output = get_calc_job_output(value)
        
        self.progress_bar.process = value
        
    def update(self):
        try:
            self.log_output.append_line(next(self._process_output))
        except (TypeError, StopIteration):
            pass
        self.progress_bar.update()

        
class CodeSubmitWidget(ipw.VBox, StagesAppStage):
    
    process = traitlets.Instance(WorkChainNode, allow_none=True)
    
    def _update_total_num_cpus(self, change):
        self.total_num_cpus.value = self.number_of_nodes.value * self.cpus_per_node.value
    
    def __init__(self, **kwargs):
        self.code_group = CodeDropdown(input_plugin='quantumespresso.pw', text="Select code", path_to_root='.')
        
        extra = {
            'style': {'description_width': '150px'},
            'layout': {'max_width': '200px'}
        }
        
        self.number_of_nodes = ipw.BoundedIntText(
            value=1, step=1, min=1,
            description="# nodes",
            disabled=False,
            **extra)
        self.cpus_per_node = ipw.BoundedIntText(
            value=1, step=1, min=1,
            description="# cpus per node",
            **extra)
        self.total_num_cpus = ipw.BoundedIntText(
            value=1, step=1, min=1,
            description="# total cpus",
            disabled=True,
            **extra)

        # Update the total # of CPUs int text:
        self.number_of_nodes.observe(self._update_total_num_cpus, 'value')
        self.cpus_per_node.observe(self._update_total_num_cpus, 'value')
                       
        self.resources = ipw.VBox(children=[
            ipw.Label("Resources:"),
            self.number_of_nodes,
            self.cpus_per_node,
            self.total_num_cpus,
        ])

        self.pseudo_family = ipw.ToggleButtons(
            options = {
                'SSSP efficiency': 'SSSP_efficiency_v1.0',
                'SSSP accuracy': 'SSSP_precision_v1.0',
            },
            description='Pseudopotential family:',
            style = {'description_width': 'initial'}
        )
        
        # Clicking on the 'submit' button will trigger the execution of the
        # submit() method.
        self.submit_button = ipw.Button(
            description='Submit',
            icon='play',
            button_style='success',
            layout=ipw.Layout(width='auto', flex="1 1 auto"),
            disabled=True)
        self.submit_button.on_click(self.submit)
        
        # The 'skip' button is only shown when the skip() method is implemented.
        self.skip_button = ipw.Button(
            description='Skip',
            icon='fast-forward',
            button_style='info',
            layout=ipw.Layout(width='auto', flex="1 1 auto"),
            disabled=True)
        if self.skip:  # skip() method is implemented
            # connect with skip_button
            self.skip_button.on_click(self.skip)  # connect with skip_button
        else:  # skip() not implemented
            # hide the button
            self.skip_button.layout.visibility = 'hidden'        
        
        # Place all buttons at the footer of the widget.
        self.buttons = ipw.HBox(children=[self.submit_button, self.skip_button])

        self.config_tabs = ipw.Tab(
            children=[self.code_group, self.pseudo_family, self.resources],
            layout=ipw.Layout(height='200px'),
        )
        self.config_tabs.set_title(0, 'Code')
        self.config_tabs.set_title(1, 'Pseudopotential')
        self.config_tabs.set_title(2, 'Compute resources')
        
        self.status = ProcessStatusWidget()
        
        self.accordion = ipw.Accordion(children=[self.config_tabs, self.status])
        self.accordion.set_title(0, 'Config')
        self.accordion.set_title(1, 'Status')
        
        self._freeze_config()
        
        super().__init__(children=[self.accordion, self.buttons], **kwargs)
        
    def request_ready_state(self):
        "Ask this stage to become active."
        self.state = StagesApp.State.READY

    @traitlets.observe('state')
    def _observe_state(self, change):
        self.skip_button.disabled = not self.state is StagesApp.State.READY
        self.submit_button.disabled = not self.state is StagesApp.State.READY

        if change['new'] == StagesApp.State.ACTIVE:
            self.accordion.selected_index = 1

        self._freeze_config(change['new'] != StagesApp.State.READY)
            
    def _freeze_config(self, value=True):
        self.code_group.dropdown.disabled = value
        self.number_of_nodes.disabled = value
        self.cpus_per_node.disabled = value
        self.pseudo_family.disabled = value
            
    @property
    def options(self):
        return {
            'max_wallclock_seconds': 3600*2,
            'resources': {
                'num_machines': self.number_of_nodes.value,
                'num_mpiprocs_per_machine': self.cpus_per_node.value}}
    
    skip = False

    def submit(self, _):
        raise NotImplementedError()
    
    
class RelaxSubmitWidget(CodeSubmitWidget):
    
    input_structure = traitlets.Instance(StructureData, allow_none=True)
    output_structure = traitlets.Instance(StructureData, allow_none=True)
    
    @traitlets.validate('input_structure')
    def _valid_input_structure(self, proposal):
        if self.output_structure is not None:
            raise traitlets.TraitError(
                "Cannot set new input_structure when output_structure is not None.")
        
        if self.state not in (StagesApp.State.INIT, StagesApp.State.READY):
            raise traitlets.TraitError(
                "Cannot set new input_structure when stage is not in state INIT or READY.")
        
        return proposal['value']
    
    @traitlets.observe('input_structure')
    def _observe_input_structure(self, change):
        if change['new'] is None:
            self.state = StagesApp.State.INIT
        else:
            self.state = StagesApp.State.READY
        self.output_structure = None

    def skip(self, _):
        self.output_structure = self.input_structure
        self.state = StagesApp.State.SUCCESS
        
    @staticmethod
    def monitor(self, process_node):
        self.status.process = process_node
        while not process_node.is_sealed:
            self.status.update()
            sleep(0.1)
        self.status.update()

        if 'output_structure' in process_node.outputs:
            self.state = StagesApp.State.SUCCESS
            self.output_structure = process_node.outputs.output_structure
        else:
            self.state = StagesApp.State.FAIL
            

    def submit(self, _):
        assert self.input_structure is not None
        
        builder = WorkflowFactory('quantumespresso.pw.relax').get_builder()
        builder.base.pw.code = self.code_group.selected_code
        builder.base.pw.parameters = DEFAULT_PARAMETERS
        builder.base.pw.metadata.options = self.options
        builder.base.kpoints_distance = Float(0.8)
        builder.base.pseudo_family = Str(self.pseudo_family.value)
        builder.structure = self.input_structure
        
        self.process = submit(builder)
        self.state = StagesApp.State.ACTIVE
        update_thread = threading.Thread(target=self.monitor, args=(self, load_node(self.process.id)))
        update_thread.start()
        

class ComputeBandsSubmitWidget(CodeSubmitWidget):
    
    input_structure = traitlets.Instance(StructureData, allow_none=True)
    output_structure = traitlets.Instance(StructureData, allow_none=True)
    
    def monitor(self, process_node):
        self.status.process = process_node
        while not process_node.is_sealed:
            self.status.update()
            sleep(0.1)
        self.status.update()
        
        print('band calc outputs', list(process_node.outputs))
        self.state = StagesApp.State.FAIL; return  # until further validated
        self.state = StagesApp.State.SUCCESS
        
    def submit(self, _):
        assert self.input_structure is not None
        
        builder = WorkflowFactory('quantumespresso.pw.bands').get_builder()

        builder.scf.pw.code = self.code_group.selected_code
        builder.scf.pw.parameters = DEFAULT_PARAMETERS
        builder.scf.pw.metadata.options = self.options
        builder.scf.kpoints_distance = Float(0.8)
        builder.scf.pseudo_family = Str(self.pseudo_family.value)

        builder.bands.pw.code = self.code_group.selected_code
        builder.bands.pw.parameters = DEFAULT_PARAMETERS
        builder.bands.pw.metadata.options = self.options
        builder.bands.pseudo_family = Str(self.pseudo_family.value)

        builder.structure = self.input_structure
        
        self.process = submit(builder)
        self.state = StagesApp.State.ACTIVE
        update_thread = threading.Thread(target=self.monitor, args=(self, load_node(self.process.id)))
        update_thread.start()
   
        
structure_widget = StructureUploadComboWidget(
    examples=[("Silicon oxide", 'common/structures/SiO2.xyz')],
    viewer=True)
relax_stage = RelaxSubmitWidget(allow_skip=True, has_next=True)
compute_bands_stage = ComputeBandsSubmitWidget()


ipw.dlink(
    (structure_widget, 'confirmed_structure'), (relax_stage, 'input_structure'),
    transform=lambda atoms: None if atoms is None else StructureData(ase=atoms))
ipw.dlink((relax_stage, 'output_structure'), (compute_bands_stage, 'input_structure'))

app = StagesApp(
    stages=[
        ('Select structure', structure_widget),
        ('Relax', relax_stage),
        ('Compute bands', compute_bands_stage)])

# FOR TESTING PURPOSES ONLY!!
structure_widget.example_widget.index = 1

app